# Gauss Elimination with Full Pivoting

## When to Use:
- When solving **ill-conditioned** systems (i.e., matrices very sensitive to small changes).
- To **maximize numerical stability** by preventing division by small pivot elements.
- **More accurate** than partial pivoting since both **rows and columns** are swapped.

### Key Point:
- We **track column swaps** separately because the final solution will be **permuted** relative to the original variables.


In [1]:
import numpy as np

def gaussEliminFullPivot(A, b):
    """
    Solves Ax = b using Gauss Elimination with Full Pivoting.
    
    Parameters:
        A : numpy.ndarray
            Coefficient matrix (n x n)
        b : numpy.ndarray
            Right-hand side vector (n)
    
    Returns:
        x : numpy.ndarray
            Solution vector
    """

    A = A.astype(float)
    b = b.astype(float)
    n = len(b)
    
    # Track column swaps
    col_order = np.arange(n)

    # -------- FORWARD ELIMINATION WITH FULL PIVOTING --------
    for pivot_row in range(n):
        # Find maximum absolute element in the submatrix A[pivot_row:, pivot_row:]
        submatrix = np.abs(A[pivot_row:, pivot_row:])
        max_idx = np.unravel_index(np.argmax(submatrix), submatrix.shape)
        max_row = pivot_row + max_idx[0]
        max_col = pivot_row + max_idx[1]

        # Swap rows
        if max_row != pivot_row:
            A[[pivot_row, max_row], :] = A[[max_row, pivot_row], :]
            b[pivot_row], b[max_row] = b[max_row], b[pivot_row]
            print(f"Row swap: {pivot_row} <-> {max_row}")

        # Swap columns (track variable permutation)
        if max_col != pivot_row:
            A[:, [pivot_row, max_col]] = A[:, [max_col, pivot_row]]
            col_order[[pivot_row, max_col]] = col_order[[max_col, pivot_row]]
            print(f"Column swap: {pivot_row} <-> {max_col}")

        # Eliminate below pivot
        for row in range(pivot_row + 1, n):
            if A[pivot_row, pivot_row] == 0:
                raise ZeroDivisionError("Zero pivot encountered!")

            multiplier = A[row, pivot_row] / A[pivot_row, pivot_row]
            A[row, pivot_row+1:] -= multiplier * A[pivot_row, pivot_row+1:]
            A[row, pivot_row] = 0  # Explicitly set for clarity
            b[row] -= multiplier * b[pivot_row]

            print(f"\nEliminating row {row} using pivot row {pivot_row}")
            print("Multiplier:", multiplier)
            print("Updated A row:", A[row])
            print("Updated b:", b[row])

    # -------- BACK SUBSTITUTION --------
    x = np.zeros(n)
    for row in range(n-1, -1, -1):
        sum_ax = np.dot(A[row, row+1:], x[row+1:])
        x[row] = (b[row] - sum_ax) / A[row, row]
        print(f"\nBack substituting for x[{row}]:", x[row])

    # Undo column permutations to restore original variable order
    x_final = np.zeros_like(x)
    for i in range(n):
        x_final[col_order[i]] = x[i]

    return x_final


## 🎯 Example: Solve Ax = b using Full Pivoting

Given:
A = [[0, 2, 3],
[4, 5, 6],
[7, 8, 9]]

b = [6, 15, 24]


This system requires column swaps because the initial pivot (A[0,0] = 0) is not ideal.


In [2]:
A = np.array([
    [0, 2, 3],
    [4, 5, 6],
    [7, 8, 9]
])

b = np.array([6, 15, 24])

x = gaussEliminFullPivot(A, b)

print("\n✅ Final Solution x:")
print(x)


Row swap: 0 <-> 2
Column swap: 0 <-> 2

Eliminating row 1 using pivot row 0
Multiplier: 0.6666666666666666
Updated A row: [ 0.         -0.33333333 -0.66666667]
Updated b: -1.0

Eliminating row 2 using pivot row 0
Multiplier: 0.3333333333333333
Updated A row: [ 0.         -0.66666667 -2.33333333]
Updated b: -2.0
Row swap: 1 <-> 2
Column swap: 1 <-> 2

Eliminating row 2 using pivot row 1
Multiplier: 0.2857142857142855
Updated A row: [ 0.          0.         -0.14285714]
Updated b: -0.42857142857142905

Back substituting for x[2]: 3.0000000000000053

Back substituting for x[1]: -1.332267629550188e-15

Back substituting for x[0]: -3.552713678800501e-15

✅ Final Solution x:
[-1.33226763e-15  3.00000000e+00 -3.55271368e-15]
